In [1]:
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup as bs
import random

In [2]:
class Digit:
    def __init__(self):
        self.choices = [x for x in range(10)]
    def guess(self):
        return random.choice(self.choices)
    def reinforce(self,value):
        self.choices.append(int(value))
    def remove(self,value):
        while int(value) in self.choices:
            self.choices.pop(self.choices.index(int(value)))
    def discourage(self,value):
        if int(value) in self.choices:
            self.choices.pop(self.choices.index(int(value)))
   

In [3]:
def enter_guess(guess):
    browser.find_by_id('guess-text').fill(guess)
    browser.find_by_id('guess-submit').click()
    
def retrieve_feedback():
    results = pd.read_html(browser.html,index_col=0)[0]
    return results

def process_answer(guess,debug=False):
    digits = [int(character) for character in guess]
    results = retrieve_feedback()
    if debug:
        print(results)
    most_recent = results.index.max()
    rdrp = results['Digit & Place'].loc[most_recent]
    rdwp = results['Digit, not Place'].loc[most_recent]
    
    if rdrp == 0 and rdwp == 0:
#         remove all digits of guess from each digits 'maybe' list and add to 'no' list
        for guess_digit in guess:
            for answer_digit in answer:
                answer_digit.remove(guess_digit)
    elif rdrp == 0:
        for index,guess_digit in enumerate(guess):
            answer[index].remove(guess_digit)
            
    if rdwp == 0:
        for g_index,guess_digit in enumerate(guess):
            for a_index,answer_digit in enumerate(answer):
                if g_index != a_index:
                    answer_digit.discourage(guess_digit)
            
    if rdwp > 0:
        for guess_digit in guess:
            for answer_digit in answer:
                answer_digit.reinforce(guess_digit)
                
    if rdrp > 0:
        for index,guess_digit in enumerate(guess):
            answer[index].reinforce(guess_digit)
            answer[index].reinforce(guess_digit)

In [4]:
# Parameters 
url = 'https://ucbe-mastermind.herokuapp.com/'
bot_name = 'beepBoop'
answer = [Digit() for x in range(4)]

In [5]:
# Visit page and enter bot_name
browser = Browser("chrome",headless=False)
browser.visit(url)
browser.find_by_id('player-name').fill(bot_name)

In [6]:
guessing = True
guess_count = 0
while guessing:
    this_guess = [str(digit.guess()) for digit in answer]
    enter_guess(this_guess)
    process_answer(this_guess)
    feedback = retrieve_feedback()
    if feedback['Digit & Place'].as_matrix()[0] == 4:
        guessing = False
    guess_count += 1 
    if guess_count > 50:
        browser.find_by_id('forfeit').click()
        
print(feedback)

    Guess  Digit & Place  Digit, not Place
23   8754              4                 0
22   1142              0                 1
21   8749              2                 1
20   5907              0                 2
19   5952              1                 0
18   9398              0                 1
17   8224              2                 0
16   2273              0                 1
15   9284              1                 1
14   9889              0                 1
13   8974              2                 1
12   1439              0                 1
11   8234              2                 0
10   7384              1                 2
9    9198              0                 1
8    7543              0                 3
7    2144              1                 0
6    1284              1                 1
5    8386              1                 0
4    9284              1                 1
3    5553              1                 0
2    1204              1                 0
1    2835  

In [7]:
pd.Series(answer[3].choices).unique()

array([6, 4, 1, 9, 3, 0, 8, 7, 2])